## Mirror SmugMug Statistics

This notebook runs queries against the `sqlite` database `mirror.db` that holds consolidated SmugMug online image metadata.

Images are here: [https://conceptcontrol.smugmug.com/](https://conceptcontrol.smugmug.com/).

In [1]:
NB. J version - notebook runtime
smoutput 9!:14''
smoutput timestamp 6!:0 ''


j903/j64avx2/windows/release-a/commercial/www.jsoftware.com/2021-12-16T15:15:09/clang-13-0-0/SLEEF=1
10 Jul 2022 17:10:43


### Load `mirrorstats` script

The [`mirrorstats.ijs`](https://github.com/bakerjd99/jacks/blob/master/mirrorxref/mirrorstats.ijs) 
script must be made available to the J `load` utility.  The [`JOD`](https://analyzethedatanotthedrivel.org/the-jod-page/) `mls` verb does this automatically. 

Alternatively you can edit the J start up script `~config/startup.ijs` and add lines like:

     buildpublic_j_ 0 : 0
     mirrorstats  ~temp/mirrorstats.ijs
     )

In [2]:
load 'mirrorstats'

NB. (mirrorstats) interface word(s): 20220710j165250
NB. -------------------------------
NB. NotDivisible  NB. albums with image counts that are not divisible by 3 and 5
NB. albdist       NB. all mean album distances km from position (x)
NB. fsd           NB. fetch sqlite dictionary array
NB. fst           NB. fetch sqlite reads table
NB. meanalbdist   NB. mean km distance of geotagged album images from (x)



### Set `mirror.db` database file location 

In [3]:
NB. set sqlite db file
(3 : 0)''
ALTMIRRORDBPATH_mirrorstats_=:''
if.     IFWIN  do. ALTMIRRORDBPATH_mirrorstats_=:'c:/SmugMirror/Documents/XrefDb/'
elseif. IFUNIX do. ALTMIRRORDBPATH_mirrorstats_=:'~/jupyter_notebooks/dbs/'
end.
if. fexist dbf=: ALTMIRRORDBPATH,MIRRORDB do. dbf else. 'db not set' end.
)
smoutput dbf

c:/SmugMirror/Documents/XrefDb/mirror.db


### Set homebase coordinates

The home coordinates are set in this fashion to avoid embedding location data in code. 

`homebase.txt` is simple text file with a single line like:
~~~
    109.375956 33.852775
~~~

Longitude and Latitude are in decimal degrees and use the sign conventions of Jean Meeus's book [*Astronomical Algorithms*](https://www.amazon.com/Astronomical-Algorithms-Jean-Meeus/dp/0943396611).

In [4]:
NB. longitude and latitude of home decimal degrees
NB. western longitudes +, northern latitudes +
(3 : 0)''
if. fexist  ALTMIRRORDBPATH,'homebase.txt' do. 
  MeeusHomeLonLat_mirrorstats_=: ".read ALTMIRRORDBPATH,'homebase.txt'
else.
  smoutput 'homebase not set using longitude=0, latitude=0'
  MeeusHomeLonLat_mirrorstats_=: 0 0
end.
)

### Set various utils

In [5]:
NB. set portable box characters for LaTeX
0 0$portchars ''

NB. cell clip format
ccf=: [ {."1 [: ": ]

### Load SQLITE data

In [6]:
NB. image columns
({."1 r)=: {:"1 r=: 'select * from OnlineImage order by UpLoadDate desc' fsd dbf

NB. album columns
({."1 r)=: {:"1 r=: 'select * from Album order by OnlineImageCount desc' fsd dbf

### Total online image count

In [7]:
'select count(*) as OnlineCount from OnlineImage' fst dbf

+-----------+
|OnlineCount|
+-----------+
|5189       |
+-----------+


### Recent album adjustments

In [8]:
NB. most recently "updated" albums
r=: 'select ImagesLastUpdated, AlbumName from Album order by LastUpdated desc limit 5' fsd dbf

NB. display with album "centroid" distance
NB. 0 distance means no geotagged images
(>&.> {:"1 r) , < ,. meanalbdist&> ;1{1{ r

+-------------------------+------------------------------+-------+
|2022-07-09T19:46:03+00:00|Idaho Instants                |117.209|
|2022-07-09T06:47:18+00:00|Caught My Eye                 |1752.08|
|2022-07-07T06:13:09+00:00|Image Hacking                 | 1311.1|
|2022-07-06T19:56:14+00:00|North Rim Monument Valley 2022|897.195|
|2022-07-06T07:01:31+00:00|Cell Phoning It In            |1312.15|
+-------------------------+------------------------------+-------+


### Recent image additions

In [9]:
NB. last 15 images loaded
;"1 ] 15 {. (10 {.&.> RealDate) ,. (<' - ') ,. OnlineImageFile

2022-07-08 - annual ketchum art festival.jpg     
2022-07-08 - mali ketchum art craft festival.jpg 
2022-07-08 - ketchum ore wagon museum.jpg        
2022-07-08 - summer sun patio umbrellas.jpg      
2022-07-07 - dreaming precariat.jpg              
2022-06-23 - old canyon guy.jpg                  
2022-06-22 - polygamy porter.jpg                 
2022-06-24 - supai tunnel kaibab trail.jpg       
2022-06-23 - velvia north rim cliff sunset.jpg   
2022-06-25 - me roosevelt point.jpg              
2022-06-27 - eratz monument valley watercolor.jpg
2022-06-27 - darktable color grade test.jpg      
2022-06-27 - monument valley three sisters.jpg   
2022-06-26 - me squinting horseshoe bend.jpg     
2022-06-25 - view from window top.jpg            


### Album statistics

In [10]:
NB. 10 largest albumns
10 {. (>AlbumName) ,. ": ,. >OnlineImageCount

Idaho Instants                479
The Way We Were               429
Cell Phoning It In            347
Weekenders                    336
Missouri Moments              315
California Captures           270
My Kids                       196
Image Hacking                 169
Montana Now and Then          166
Panoramas                     161


In [11]:
NB. album image counts
dstat OnlineImageCount

sample size:          79
minimum:               0
maximum:             479
1st quartile:         17
2nd quartile:         48
3rd quartile:        109
first mode:           21
first antimode:      479
mean:            79.5949
std devn:        96.5491
skewness:         2.2503
kurtosis:         8.2612


### Images per year

My image dates are a mixture of EXIF dates and manually assigned dates. The manually assigned dates are rarely accurate and reflect my best guess as to when a picture was taken. Many images do not have dates. They show up in year 0.

In [12]:
NB. images per year - year 0 means no date
70 list ',' ,.~  ": rf=: \:~ |: yic=: > freq 0 {"1 rdt=: itYMDhms RealDate

2022 240, 2021 163, 2020  98, 2019 446, 2018 294, 2017 227, 2016 298, 
2015 207, 2014  98, 2013 172, 2012 178, 2011 217, 2010 152, 2009 144, 
2008 180, 2007 284, 2006 166, 2005 120, 2004 110, 2003  79, 2002 118, 
2001 135, 2000  33, 1999   6, 1996   1, 1995  19, 1994   8, 1993   8, 
1992  18, 1991  12, 1990   4, 1989  14, 1988   6, 1987  10, 1986   6, 
1985   4, 1984   2, 1983  42, 1982   4, 1981  37, 1980  17, 1979  90, 
1978  27, 1977  10, 1976  20, 1975   8, 1974  22, 1973   4, 1972   5, 
1971  23, 1970  33, 1969  22, 1968  45, 1967  32, 1966  24, 1965   5, 
1964  13, 1963   8, 1962  20, 1961   6, 1960   7, 1959  15, 1958  13, 
1957   7, 1956   6, 1955  11, 1954  10, 1953   4, 1952   7, 1951   3, 
1950   9, 1949  17, 1948  10, 1947   3, 1946   5, 1945   2, 1944   2, 
1943   1, 1942   1, 1941   7, 1940   6, 1939   2, 1938   3, 1937   2, 
1936   5, 1935   4, 1934   3, 1933   1, 1932   1, 1931   4, 1930   1, 
1929   1, 1928   1, 1927   2, 1925   2, 1924   1, 1920   2, 1919   2, 
1917  

Images per year statistics since the year 2000. It was around 2000 that I started saving digital image files.

In [13]:
NB. images per year statistics for years after 1999
dstat (1 {"1 rf) #~ 1999 < 0 {"1 rf

sample size:          23
minimum:              33
maximum:             446
1st quartile:        118
2nd quartile:        166
3rd quartile:        227
first mode:           98
first antimode:      240
mean:            180.826
std devn:        90.1119
skewness:         1.0635
kurtosis:         4.4468


### Uploads per year

I often upload old images that get counted in prior years. These counts reflect actual database inserts. Note the count of 724 in 2019. I did not insert that many images in 2019. The upload count probably reflects reorganizing existing images. The count of 1531 in 2009 marks the year I transferred my online images to SmugMug.

In [14]:
70 list ',' ,.~  ": uf=: \:~ |: upc=: > freq 0 {"1 udt=: itYMDhms UploadDate

2022  250, 2021  203, 2020  109, 2019  724, 2018  352, 2017  319, 
2016  362, 2015  222, 2014  102, 2013  256, 2012  233, 2011  314, 
2010  212, 2009 1531,                                             


In [15]:
NB. uploads per year statistics - less outliers > 600
dstat (1 {"1 uf) #~ 600 > 1 {"1 uf

sample size:          12
minimum:             102
maximum:             362
1st quartile:      207.5
2nd quartile:      241.5
3rd quartile:      316.5
first mode:          250
first antimode:      250
mean:              244.5
std devn:        84.2016
skewness:        _0.2998
kurtosis:         2.2336


### *Flarp* rate

The *flarp rate* gives a rough indication of how much image rework and reloading I do. It shows up as a discrepancy between year counts and upload counts. 

In [16]:
smoutput flc=: (0{upc) ,: (1{upc) - 1{(\:~)&.|: yic #"1~ (0{yic) e. 0{upc

2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010 2009
  10   40   11  278   58   92   64   15    4   84   55   97   60 1387


Flarp statistics excluding the first upload year.

In [17]:
dstat }: 1{flc

sample size:          13
minimum:               4
maximum:             278
1st quartile:         13
2nd quartile:         58
3rd quartile:         88
first mode:           10
first antimode:       10
mean:            66.7692
std devn:        70.9485
skewness:         2.1152
kurtosis:         7.2521


### `NotDivisible` counts

`NotDivisible` counts how many images must be added to a gallery so that the total number of gallery images is evenly divisible by 3 and 5. The iPhone SmugMug app displays images in rows of 3 (portrait) or 5 (landscape) and I like completely filled rows. When the total is evenly divisible by 15 = 3 * 5 both portrait and landscape rows are filled.

In [18]:
10 {. NotDivisible dbf

+-------+-+-+--+--------------------+
|[Count]|3|5|15|[Album Name]        |
+-------+-+-+--+--------------------+
|479    |1|1|1 |Idaho Instants      |
+-------+-+-+--+--------------------+
|133    |2|2|2 |From Hazel's Albums |
+-------+-+-+--+--------------------+
|161    |1|4|4 |Panoramas           |
+-------+-+-+--+--------------------+
|142    |2|3|8 |Caught My Eye       |
+-------+-+-+--+--------------------+
|7      |2|3|8 |Does Not Fit        |
+-------+-+-+--+--------------------+
|7      |2|3|8 |Test Patterns       |
+-------+-+-+--+--------------------+
|7      |2|3|8 |Partial Restorations|
+-------+-+-+--+--------------------+
|22     |2|3|8 |To Much Information |
+-------+-+-+--+--------------------+
|7      |2|3|8 |Fake Pixels         |
+-------+-+-+--+--------------------+


### Color keywords

Keywords starting with `0_` are dominant color keys assigned by the Python `SmugPyter` program. 

In [19]:
NB. keyword columns
({."1 r)=: {:"1 r=: 'select * from ImageKeywordXr' fsd dbf

In [20]:
topbotk=: 4 : 0
NB.topbotk v-- (x) top and bottom (y) keys
kf=: ofreq y
kf=: (' ' ,. >;0{kf) ,.~ ":>,.;1{kf
(x {. kf) ,. |. (-x) {. kf
)

**Note:** the abundance of neutral, (grey and near grey), colors. Well exposed and color balanced color images tend, on average, to be neutral. This is why photographers use grey cards and color balancing software picks neutrals.

In [21]:
NB. top/bottom 20 color keywords
ColorKeys=: Keyword #~ ('0_'&-:)@(2&{.)&> Keyword
20 topbotk ColorKeys

529 0_darkslategrey         1 0_forestgreen         
375 0_black                 1 0_aquamarine          
265 0_darkolivegreen        1 0_purple              
236 0_steelblue             1 0_magenta             
234 0_dimgrey               1 0_lightyellow         
209 0_darkgrey              1 0_lawngreen           
192 0_rosybrown             1 0_lavenderblush       
178 0_gainsboro             1 0_indigo              
178 0_lightsteelblue        1 0_lightgreen          
173 0_lightgrey             1 0_springgreen         
172 0_cornflowerblue        1 0_cyan                
168 0_grey                  1 0_hotpink             
168 0_tan                   1 0_darkmagenta         
121 0_silver                1 0_mediumspringgreen   
120 0_lavender              1 0_floralwhite         
 93 0_lightslategrey        1 0_cornsilk            
 91 0_peru                  1 0_mediumorchid        
 90 0_sienna                2 0_greenyellow         
 82 0_skyblue               2 0_lime          

### Print size keywords

In [22]:
NB. top/bottom 20 print size keywords
SizeKeys=: ColorKeys -.~ Keyword #~ ({.&> Keyword) e.&> <'0123456789'
NB. remove odd keys that do not follow name conventions
NssKeys=: <;._1 ' 0z0 0z1'
SizeKeys=: SizeKeys -. (<;._1 ' 50th'),NssKeys
20 topbotk SizeKeys

943 4x6       1 2.25x4  
908 5x7       1 10x16   
550 8x12      1 8x40    
305 10x15     1 12x30   
274 5x6.7     1 6x24    
269 6x8       1 16x24   
198 3x4       1 3x3     
152 2.5x3.5   1 5x8     
136 4x5       1 5.5x14  
 93 5x5       1 9x21    
 67 6x10      1 4x5.3   
 61 3x5       1 2x2.5   
 55 8x10      2 10x40   
 52 2x3       2 16x20   
 28 4x4       2 12x24   
 27 10x14     2 6x15    
 25 3.5x5     2 10x13   
 25 8x8       2 11x28   
 15 6x6       2 11x16   
 15 5x6       2 9x16    


In [23]:
NB. size keys that indicate not enough pixels or nonstandard aspect ratio
NssKeys ,. <"0 +/"1 NssKeys =/ Keyword 

+---+---+
|0z0|258|
+---+---+
|0z1|475|
+---+---+


### Descriptive keywords

All keywords that are not color or size keys.

In [24]:
DescKeys=: Keyword -. ColorKeys,SizeKeys,NssKeys
20 topbotk DescKeys

1316 geotagged                     1 rafting                    
1036 usa                           1 prayer                     
 546 mali                          1 tooth                      
 394 me                            1 saber                      
 246 idaho                         1 columns                    
 228 park                          1 remnants                   
 205 helen                         1 pictographs                
 202 iphone                        1 skeleton                   
 187 lake                          1 sizes                      
 173 river                         1 rafts                      
 173 canada                        1 dell                       
 168 missouri                      1 large                      
 150 jacob                         1 josie                      
 143 museum                        1 powers                     
 128 ontario                       1 fremont                    
 127 montana             

### Panorama statistics

I have enjoyed making panoramas for years. I consider any image with an *`aspect ratio >: 2`* a panorama.  Many of my panoramas have nonstandard ratios, i.e. they are not simple `2x1`, `3x1`, `4x1`, `5x1`  ... ratios and consequently do not get assigned a size key. 

In [25]:
NB. rounded image aspect ratios - aspect ratio is always >: 1
ratios=: 0.005 round %/"1 \:"1~ OriginalHeight ,. OriginalWidth

NB. aspect ratio counts
PanAspects=: >: i. >: >. >./ratios
smoutput RatioCounts=: ;"1 (2 <\ PanAspects) ,. <"0 }. }: PanAspects histogram2 ratios

1 2 4959
2 3  145
3 4   62
4 5   19
5 6    4
6 7    0


### Total number of panoramas with *Aspect Ratio >: 2*

**Note:** This simple criteria of picking images based on aspect ratio misses some panoramas. Some images I have built from more than one image have standard aspect ratios.

In [26]:
+/pancnts=: }. 2 {"1 RatioCounts

230


### Online panorama ratios

In [27]:
NB. percentage of panoramic images by aspect ratio - last % is overall
smoutput panper=: (}. 0 2 {"1 RatioCounts) ,. 0.01 round 100 * pancnts % #ratios
smoutput '-' ,: '%' ,~ ": }. +/panper

2 145 2.79
3  62 1.19
4  19 0.37
5   4 0.08
6   0    0
---------
230 4.43%


### Most extreme panoramas *Aspect Ratio >: 5*

In [28]:
NB. most extreme panoramas, i.e. aspect ratios >: 5
80 ccf (OnlineImageFile ,. RealDate ,. Keywords) #~ 5 <: ratios

+---------------------------------------+-------------------+-------------------
|pyramid lake panorama.jpg              |2021-05-04T09:37:47|0_dodgerblue; 8x40;
+---------------------------------------+-------------------+-------------------
|adndlbanner01.jpg                      |                   |0_gold; 0z0; adndlb
+---------------------------------------+-------------------+-------------------
|jodlittlecubebanner.jpg                |                   |0_lightgrey; 0z0; j
+---------------------------------------+-------------------+-------------------
|alexandra bridge ottawa [407055261].jpg|2007-02-28T14:10:00|0_gainsboro; 0z1; a
+---------------------------------------+-------------------+-------------------


### Recently uploaded panoramas

In [29]:
NB. most recent panoramas
80 ccf ,. 5 {. (OnlineImageFile ,. RealDate ,. Keywords) #~ 2 <: ratios

+----------------------------+-------------------+------------------------------
|me roosevelt point.jpg      |2022-06-25T11:30:06|0_dimgrey; 4x10; arizona; geot
+----------------------------+-------------------+------------------------------
|view from window top.jpg    |2022-06-25T10:45:01|0_darkslateblue; 10x20; arizon
+----------------------------+-------------------+------------------------------
|pure point imperial.jpg     |2022-06-25T10:19:27|0_cornflowerblue; 10x25; arizo
+----------------------------+-------------------+------------------------------
|north rim window diptych.jpg|2022-06-25T13:20:21|0_darkolivegreen; 6x18; diptyc
+----------------------------+-------------------+------------------------------
|on the way to page.jpg      |2022-06-26T10:39:02|0_skyblue; 8x20; page; the; wa
+----------------------------+-------------------+------------------------------


### GPS distances

In [30]:
NB. distances in km from near to far excluding (0,0) lb
NB. convert to Meeus lb convention for (earthdist)
plb=: (i. #Longitude) ,. (-Longitude) ,. Latitude
plb=: |: plb #~ -.0 0 -:"1 ] 1 2 {"1 plb
plb=: /:~ (0{plb) ,.~ MeeusHomeLonLat earthdist 1 2{plb

In [31]:
NB. format image distance
fmdist=: {{;"1 ":&.> (<"0 ] 0.01 round 0 {"1 y) ,. (<' - ') ,. (1 {"1 y){OnlineImageFile}}

In [32]:
NB. 10 nearest geotagged images km
fmdist 10{.plb

0.11 - neighborhood intersection park.jpg           
0.12 - road end promises.jpg                        
0.16 - not so burning bush.jpg                      
0.32 - meridian irrigation canal.jpg                
0.51 - mashed squirrel roadkill.jpg                 
0.57 - school playground map.jpg                    
0.64 - more mcmansion construction.jpg              
0.64 - new meridian neighborhood.jpg                
0.64 - territory drive house construction august.jpg
0.64 - territory drive house building.jpg           


In [33]:
NB. 10 furthest geotagged images km
fmdist |. _10{.plb

15830.9 - pishi on pishi [47018603].jpg                      
15487.7 - air malawi [42502530].jpg                          
15477.3 - victoria falls hotel distance marker [36406766].jpg
15476.9 - me livingstone statue [19983766].jpg               
15476.8 - vic falls train station [51228396].jpg             
15476.6 - curio market from balloon [49529252].jpg           
15476.3 - mali sprayview hotel [51064919].jpg                
15467.2 - me nude kafunta chalet [1157046406].jpg            
15467.2 - kafunta lodge veranda [2486888].jpg                
15466.7 - me livingstone airport [47004492].jpg              


In [34]:
NB. image distance statistics
dstat 0 {"1 plb

sample size:        1316
minimum:          0.1089
maximum:         15830.9
1st quartile:    434.064
2nd quartile:    904.069
3rd quartile:    2252.84
first mode:      849.169
first antimode:   0.1089
mean:            1751.63
std devn:        2471.97
skewness:         3.2823
kurtosis:        15.8789


In [35]:
NB. 1s mark panoramas
pbits=: (}.PanAspects) <:"0 1 %/"1 \:"1~ OriginalHeight ,. OriginalWidth

NB. panorama image distances
gtp=: plb #~ (1 {"1 plb) e. I. 0{pbits 

In [36]:
NB. 10 nearest panoramas
fmdist 10 {. gtp

0.64 - more mcmansion construction.jpg                
0.64 - new meridian neighborhood.jpg                  
0.64 - territory drive house construction august.jpg  
0.64 - territory drive house building.jpg             
3.73 - joplin cemetery collage.jpg                    
4.32 - scentsy amphitheater grounds.jpg               
8.94 - boise mall panorama.jpg                        
13.99 - owyhee sign.jpg                               
19.04 - treasure valley inversion bogus basin road.jpg
25.5 - boxing day bogus basin.jpg                     


In [37]:
NB. 10 furthest panoramas
fmdist |. _10 {. gtp

15440.4 - ndeke hotel sign [37985773].jpg            
8447.83 - lab south view 12x4 [5037801].jpg          
4629.83 - me spittal pond bermuda [278786543].jpg    
4552.7 - waikiki beach diamond head panorama.jpg     
4551.05 - honolulu from diamond head panorama.jpg    
4550.86 - diamond head caldera interior.jpg          
4485.65 - mauna kea sunset summit panorama.jpg       
3221.21 - 132 topham terrace orleans white winter.jpg
3219.14 - petrie island river [98449654].jpg         
3203.8 - alexandra bridge ottawa [407055261].jpg     


In [38]:
NB. mean panorama distance
dstat 0 {"1 gtp

sample size:         109
minimum:          0.6357
maximum:         15440.4
1st quartile:    239.321
2nd quartile:    541.554
3rd quartile:    1256.63
first mode:       0.6357
first antimode:   3.7257
mean:            1209.72
std devn:        1939.36
skewness:         4.4098
kurtosis:        29.4358


In [39]:
NB. mean album geotagged image distances
mad=: albdist 0

In [40]:
NB. album "centroid" distance statistics
dstat ;0 {"1 mad

sample size:          61
minimum:           4.051
maximum:         15482.4
1st quartile:    847.156
2nd quartile:    1955.79
3rd quartile:    3189.11
first mode:      930.509
first antimode:  930.509
mean:            3002.29
std devn:        3345.49
skewness:         1.9391
kurtosis:         6.2261


In [41]:
NB.format album distances.
fmtad=:{{('8.2'&(8!:2) ,. km) ,. ' - ' ,"1 alb [ 'km alb'=. |: x {. y}}

In [42]:
NB. 10 nearest albums
10 fmtad /:~ mad

    4.05 - Improvised Imaging Devices
    8.63 - Camera Tests              
  117.21 - Idaho Instants            
  305.60 - Anne and Reza Visit       
  487.48 - Collages                  
  490.53 - Montana Now and Then      
  503.38 - Logos Screenshots Covers  
  535.33 - North by Northwest        
  546.91 - Tetons Yellowstone 2013   
  610.78 - From Hazel's Albums       


In [43]:
NB. 10 furthest albums
10 fmtad \:~ mad

15482.40 - Zambia Eclipse Trip           
11660.41 - Iran 1960's                   
11272.49 - Ghana 1970's                  
11225.88 - ACS School Trips 1960's       
10963.97 - Beirut Lebanon 1960's         
 8457.47 - South America 1979            
 8409.77 - Enewetak Atoll 1980's         
 6347.19 - Diving at Bellairs Barbados BW
 6346.13 - Barbados Late 1970's          
 6046.94 - GPS Extremes                  


### All done - thanks for playing!